#### Prediction Model to predict the staff being present

In [1]:
import os
import numpy as np
import cv2 # Extract images from video
import tensorflow as tf # Tensorflow stuff
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import pandas as pd

#### Training Data

In [2]:
df = pd.read_csv('./data/prediction/train.csv')
X_img_name = df['img_number']
X = []
y = df['truth']

for i in range(len(X_img_name)):
    im = cv2.imread('./data/prediction/{}'.format(str(X_img_name[i])))
    X.append(im)

X = np.array(X)
y = np.array(y)
y = y.reshape((74, 1))

print("x: " + str(X.shape) + "  y: " + str(y.shape))

x: (74, 720, 960, 3)  y: (74, 1)


In [3]:
# Resize, normalising X and preprocess for ResNet50
X = X.astype('float32')
X /= 255

X = tf.image.resize(images=X, size=[240, 320])
X = tf.keras.applications.resnet50.preprocess_input(X)
X = X.numpy()

# Split x and y into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

#### Model Preparation

In [4]:
# PRETRAINED RESNET MODEL WITHOUT FC LAYER
base_model = tf.keras.applications.ResNet50(input_shape=(240, 320, 3), include_top=False, weights='imagenet')

# Freeze trainable layers
base_model.trainable = False

output = base_model.output
output = tf.keras.layers.Flatten() (output)

# Add new FC Layers to the output that can predict the XY coordinate and prediction
output = tf.keras.layers.Dense(128, activation="relu") (output)
output = tf.keras.layers.Dense(64, activation="relu") (output)
output = tf.keras.layers.Dense(32, activation="relu") (output)
output = tf.keras.layers.Dense(1, activation="sigmoid") (output) # sigmoid because binary image classification to detect the presence of tag

model = tf.keras.models.Model(inputs=base_model.input, outputs=output)
# model.summary()

In [10]:
# MODEL PARAMETERS
EPOCHS = 40
BATCH_SIZE = 16
OPTIMIZER = tf.optimizers.Adam()
LOSS_FUNC = tf.keras.losses.binary_crossentropy

CALLBACKS = [
    tf.keras.callbacks.ModelCheckpoint(filepath="./model/prediction/prediction_model", monitor='accuracy', mode='max', verbose=False, save_best_only=True)
        ]

# Initialize optimizer, compile and start fine tuning the model
model.compile(optimizer=OPTIMIZER, loss=LOSS_FUNC, metrics=['accuracy'])

# Fit the model
history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, callbacks=CALLBACKS, epochs=EPOCHS, verbose=1, validation_data=(X_test, y_test))

Epoch 1/40
3/4 [=====================>........] - ETA: 0s - loss: 2.6968 - accuracy: 0.5000

INFO:tensorflow:Assets written to: ./model/prediction\prediction_model\assets


INFO:tensorflow:Assets written to: ./model/prediction\prediction_model\assets


4/4 [==============================] - 16s 5s/step - loss: 3.2099 - accuracy: 0.4576 - val_loss: 3.3207 - val_accuracy: 0.5333
Epoch 2/40
3/4 [=====================>........] - ETA: 0s - loss: 1.7978 - accuracy: 0.5000

INFO:tensorflow:Assets written to: ./model/prediction\prediction_model\assets


...

INFO:tensorflow:Assets written to: ./model/prediction\prediction_model\assets


4/4 [==============================] - 14s 5s/step - loss: 0.3291 - accuracy: 0.8983 - val_loss: 0.5278 - val_accuracy: 0.6667
Epoch 21/40
4/4 [==============================] - 0s 52ms/step - loss: 0.3339 - accuracy: 0.8814 - val_loss: 0.5358 - val_accuracy: 0.6000
Epoch 22/40
4/4 [==============================] - 0s 49ms/step - loss: 0.2988 - accuracy: 0.8475 - val_loss: 0.5599 - val_accuracy: 0.6000
Epoch 23/40
4/4 [==============================] - 0s 55ms/step - loss: 0.3109 - accuracy: 0.8475 - val_loss: 0.8408 - val_accuracy: 0.6667
Epoch 24/40
4/4 [==============================] - 0s 51ms/step - loss: 0.3634 - accuracy: 0.8305 - val_loss: 0.6597 - val_accuracy: 0.7333
Epoch 25/40
4/4 [==============================] - 0s 51ms/step - loss: 0.3477 - accuracy: 0.8475 - val_loss: 0.7471 - val_accuracy: 0.6667
Epoch 26/40
4/4 [==============================] - 0s 51ms/step - loss: 0.3591 - accuracy: 0.8644 - val_loss: 0.6587 - val_accuracy: 0.6667
Epoch 27/40
4/4 [================

INFO:tensorflow:Assets written to: ./model/prediction\prediction_model\assets


INFO:tensorflow:Assets written to: ./model/prediction\prediction_model\assets


4/4 [==============================] - 14s 5s/step - loss: 0.4128 - accuracy: 0.7627 - val_loss: 0.4867 - val_accuracy: 0.6667
Epoch 34/40
4/4 [==============================] - 1s 51ms/step - loss: 0.3153 - accuracy: 0.8475 - val_loss: 0.5064 - val_accuracy: 0.7333
Epoch 35/40
4/4 [==============================] - 0s 50ms/step - loss: 0.2617 - accuracy: 0.9322 - val_loss: 0.7083 - val_accuracy: 0.6667
Epoch 36/40
4/4 [==============================] - 0s 51ms/step - loss: 0.3457 - accuracy: 0.9153 - val_loss: 0.5022 - val_accuracy: 0.7333
Epoch 37/40
4/4 [==============================] - 0s 53ms/step - loss: 0.3296 - accuracy: 0.8305 - val_loss: 0.4931 - val_accuracy: 0.7333
Epoch 38/40
4/4 [==============================] - 0s 49ms/step - loss: 0.2887 - accuracy: 0.8814 - val_loss: 0.8586 - val_accuracy: 0.6667
Epoch 39/40
4/4 [==============================] - 0s 53ms/step - loss: 0.2941 - accuracy: 0.8475 - val_loss: 0.4936 - val_accuracy: 0.6667
Epoch 40/40
4/4 [================

In [11]:
# Save Model
model.save("./model/prediction/prediction_model_2")

INFO:tensorflow:Assets written to: ./model/prediction/prediction_model_2\assets


INFO:tensorflow:Assets written to: ./model/prediction/prediction_model_2\assets
